In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
# Define Directories
working_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Working_files'
output_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Reports'

# Get current month and year
current_date = datetime.now().strftime("%Y-%m-%d")

# Find the current month's files 
files = [f for f in os.listdir(working_dir) if f.endswith(".xlsx")]
print(files)

# Read and combine data
all_data = []
for file in files:
    file_path = os.path.join(working_dir,file)
    try:
        df = pd.read_excel(file_path, dtype={'Batch': str})
        df['Source File'] = file
        all_data.append(df[['Source File','Plant', 'Storage Location', 'Material', 'Description', 'Batch', 'Qty_SAP', 'Qty_WH', 'Delta', 'QC Hold Qty PSS', 'Hold Reason ES3',
                            'Manufacture Date', 'Critical Shelf Life Date', 'Expiration date', 'Number of Summers', 'Unit Value', 'Pallet number']])
    except Exception as e:
        print(f"Failed to process {file}: {e}")

In [5]:
# Combine into single DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Group by key fields and find constant deltas
    summary = (combined_df.groupby(['Plant', 'Storage Location', 'Material', 'Description', 'Batch'])
                           .agg(Min_Delta = ('Delta', 'min'),
                                Max_Delta = ('Delta', 'max' ),
                                Occurances = ('Delta', 'count'))
                           .reset_index())

    # Filter for consistent deltas (min = max) appearing at least 3 times, excluding zero deltas
    consistent_deltas = summary[
        (summary['Min_Delta'] == summary['Max_Delta']) & # Delta is constant
        (summary['Min_Delta'] != 0) &                    # Exclude zero deltas
        (summary['Occurances'] >=3)                      # Appear in 3+ files
    ]
    # Ouput to Excel
    output_file = os.path.join(output_dir, f"constant_delta_report_{current_date}.xlsx")
    
    # Create and Excel writer object
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Write the constan delta sheet
        consistent_deltas.to_excel(writer, sheet_name="Constant Delta", index=False)

        # Write separate sheets per warehouse
        for warehouse in ['BP01', 'BP02', 'BP04', 'BP07']:
            warehouse_data = combined_df[combined_df['Plant'] == warehouse]
            if not warehouse_data.empty:
                warehouse_data = warehouse_data.sort_values(by='Material') # Sort by Material
                warehouse_data.to_excel(writer, sheet_name=warehouse, index=False)

    print(f"Report generated: {output_file}")
else:
    print("No data to process")
